In [2]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

In [35]:
capture = cv.VideoCapture("volleyball_match.mp4") 

fullbody_cascade = cv.CascadeClassifier("haar_full.xml")

while True:
    
    isTrue , frame = capture.read()
    
    blur = cv.GaussianBlur(frame , (5,5) , cv.BORDER_DEFAULT)
    
    gray = cv.cvtColor(blur , cv.COLOR_BGR2GRAY)

    magnitude = cv.Canny(blur , 125 , 175)
    
    ret , thresh = cv.threshold(magnitude , 100 , 255 ,  cv.THRESH_BINARY)
    
    contours , hierarchies = cv.findContours(thresh , cv.RETR_LIST , cv.CHAIN_APPROX_SIMPLE)
    
    for contour in contours :
        
        perimeter = cv.arcLength(contour, True)
        if(perimeter == 0) :
            continue  
        
        area = cv.contourArea(contour)
        circularity = 4 * 3.14 * (area / (perimeter * perimeter))
        
        if(area > 100  or area < 95  or circularity < 0.75) :
            continue
            
        x, y, w, h = cv.boundingRect(contour)
        cv.rectangle(frame, (x-10, y-10), (x + w + 10, y + h + 10), (2, 255 , 0), thickness = 2)
        cv.putText(frame, "VolleyBall", (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
              
      
    cv.imshow("VolleyBall Tracker" , frame) 
    
    
    if(not isTrue) :
        break 
    if cv.waitKey(15) & 0xFF == ord('d') : 
        break
        
capture.release()  
cv.destroyAllWindows() 

In [36]:
import cv2 as cv
import numpy as np

capture = cv.VideoCapture("volleyball_match.mp4")

fullbody_cascade = cv.CascadeClassifier("haar_full.xml")

isTrue, prev_frame = capture.read()
prev_gray = cv.cvtColor(prev_frame, cv.COLOR_BGR2GRAY)
prev_blur = cv.GaussianBlur(prev_gray, (5, 5), cv.BORDER_DEFAULT)

while True:
    isTrue, frame = capture.read()
    if not isTrue:
        break

    blur = cv.GaussianBlur(frame, (5, 5), cv.BORDER_DEFAULT)
    gray = cv.cvtColor(blur, cv.COLOR_BGR2GRAY)
    diff = cv.absdiff(prev_blur, gray)
    _, motion_mask = cv.threshold(diff, 20, 255, cv.THRESH_BINARY)
    motion_mask = cv.dilate(motion_mask, None, iterations=2) 
    red_count = 0
    yellow_count = 0
    bodies = fullbody_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3)

    for (x, y, w, h) in bodies:
        motion_roi = motion_mask[y:y+h, x:x+w]
        if cv.countNonZero(motion_roi) < 100:
            continue

        person_roi = frame[y:y+h, x:x+w]
        hsv_roi = cv.cvtColor(person_roi, cv.COLOR_BGR2HSV)
        mask_red = cv.inRange(hsv_roi, (0, 70, 50), (10, 255, 255)) 
        mask_yellow = cv.inRange(hsv_roi, (20, 100, 100), (35, 255, 255))
        red_pixels = cv.countNonZero(mask_red)
        yellow_pixels = cv.countNonZero(mask_yellow)

        if red_pixels > yellow_pixels and red_pixels > 100:
            red_count += 1
            color = (0, 0, 255)
            label = "Red Player"
        elif yellow_pixels > red_pixels and yellow_pixels > 100:
            yellow_count += 1
            color = (0, 255, 255)
            label = "Yellow Player"
        else:
            continue

        cv.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv.putText(frame, label, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    cv.putText(frame, f"Red Players: {red_count}", (20, 30),
               cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    cv.putText(frame, f"Yellow Players: {yellow_count}", (20, 60),
               cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
    magnitude = cv.Canny(blur, 125, 175)
    combined = cv.bitwise_and(magnitude, motion_mask)

    ret, thresh = cv.threshold(combined, 100, 255, cv.THRESH_BINARY)
    contours, hierarchies = cv.findContours(thresh, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        perimeter = cv.arcLength(contour, True)
        if perimeter == 0:
            continue

        area = cv.contourArea(contour)
        circularity = 4 * 3.14 * (area / (perimeter * perimeter))

        if area > 100 or area < 95 or circularity < 0.75:
            continue

        x, y, w, h = cv.boundingRect(contour)
        cv.rectangle(frame, (x - 10, y - 10), (x + w + 10, y + h + 10), (0, 255, 0), thickness=2)
        cv.putText(frame, "VolleyBall", (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv.imshow("VolleyBall Tracker", frame)

    prev_blur = gray.copy()

    if cv.waitKey(15) & 0xFF == ord('d'):
        break

capture.release()
cv.destroyAllWindows()
